In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!pip install openvino-dev
#!pip install onnxruntime

In [3]:
from IPython.display import Markdown, display
import onnxruntime

In [4]:
import sys,os
import torch
sys.path.append(os.path.abspath('../models'))
from tinyism import tinyModel

In [5]:
posencoding=True
model=tinyModel(posEncoding=posencoding)

In [6]:
 weight="../weights/resnet18_inst.pth"
 model.load_state_dict(torch.load(weight,map_location=torch.device('cpu')))
 model.eval()

tinyModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [40]:
    sys.path.append(os.path.abspath('../datasets'))
    from data_loader import preprocess
    import cv2
    img=cv2.imread("../test.png")
    img=cv2.resize(img,(512,512))
    x=preprocess((torch.from_numpy(img).float())).unsqueeze(0)

In [41]:
torchout=model(x)
x.shape,torchout[0].shape,torchout[1].shape

(torch.Size([1, 3, 512, 512]), torch.Size([4]), torch.Size([4, 64, 64]))

In [9]:
input_names=['images']
output_names = ['output0', 'output1'] 
dynamic = {'images': {0: 'batch'}}  
dynamic['output0'] = {0: 'instances'} 
dynamic['output1'] = {0: 'instances'}  # shape(1,insts,64,64)


torch.onnx.export(model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "tinynet.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = input_names,   # the model's input names
                  output_names = output_names, # the model's output names
                  dynamic_axes=dynamic
                  )

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/einops/einops.py:204: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  inferred_length: int = length // known_product
/home/ec2-user/SageMaker/tiny-instance-segmentation/models/tinyism.py:42: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  if torch.tensor(bidx.size())>maxdets:
/home/ec2-user/SageMaker/tiny-instance-segmentation/models/tinyism

In [42]:
ort_session = onnxruntime.InferenceSession("tinynet.onnx")

In [43]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [44]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

In [45]:
ort_outs[0].shape,ort_outs[1].shape

((4,), (4, 64, 64))

In [46]:
input_image=to_numpy(x)
torchout1=to_numpy(torchout[0])
torchout2=to_numpy(torchout[1])
ort_outs[0].shape,ort_outs[1].shape,torchout1.shape,torchout2.shape

((4,), (4, 64, 64), (4,), (4, 64, 64))

In [47]:
import numpy as np
np.allclose(torchout1, ort_outs[0], rtol=1e-03, atol=1e-05),np.allclose(torchout2, ort_outs[1], rtol=1e-03, atol=1e-05)

(True, True)

In [16]:
# Construct the command for Model Optimizer.
from pathlib import Path
onnx_path="tinynet.onnx"
model_path="irmodel"
model_path = Path(model_path)
ir_path = model_path.with_suffix(".xml")
IMAGE_HEIGHT=512
IMAGE_WIDTH=512
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --input_shape "[1,3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --data_type FP32
                 --output_dir "{model_path}"
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert the ONNX model to OpenVINO:


`mo --input_model "tinynet.onnx" --input_shape "[1,3, 512, 512]" --data_type FP32 --output_dir "irmodel"`

In [18]:
print("Exporting ONNX model to IR... This may take a few minutes.")
mo_result = %sx $mo_command
print("\n".join(mo_result))

Exporting ONNX model to IR... This may take a few minutes.
[ WARNING ]  Use of deprecated cli option --data_type detected. Option use in the following releases will be fatal. 
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/ec2-user/SageMaker/tiny-instance-segmentation/notebooks/irmodel/tinynet.xml
[ SUCCESS ] BIN file: /home/ec2-user/SageMaker/tiny-instance-segmentation/notebooks/irmodel/tinynet.bin


In [19]:
ir_path=Path("irmodel/tinynet.xml")

In [48]:
from openvino.runtime import Core
ie = Core()

In [49]:
model_ir = ie.read_model(model=ir_path)
compiled_model_ir = ie.compile_model(model=model_ir, device_name="CPU")

In [50]:
# Get input and output layers.
output_layer_ir0 = compiled_model_ir.output(0)
output_layer_ir1 = compiled_model_ir.output(1)
output_layer_ir0

<ConstOutput: names[output0] shape[..4096] type: i64>

In [58]:
res_ir0 = compiled_model_ir([input_image])[output_layer_ir0]
res_ir1 = compiled_model_ir([input_image])[output_layer_ir1]
res_ir0.shape,res_ir1.shape

((4,), (4, 64, 64))

In [57]:
np.allclose(res_ir0,torchout1,rtol=1e-04, atol=1e-05),np.allclose(res_ir1,torchout2,rtol=1e-03, atol=1e-05)

(True, True)